## 22.4 Optimise

I've shown you how to solve constraint satisfaction problems on sequences
with backtracking. It shouldn't be a surprise that backtracking can also
solve optimisation problems.
Instead of collecting all solutions in a list or set,
the algorithm keeps only the best solution found so far.
Let's see an example.

### 22.4.1 The problem

I take the earlier problem of finding sequences of numbers from 1 to *n*
that satisfy range and parity constraints, and add an optimisation criterion:
we want a sequence with a lowest sum.
For example, for *n* = 4, the best sequence is (1, 4) because its sum 5
is lowest among all other sequences that satisfy both constraints.

An optimisation problem asks to find a solution that minimises or maximises
some value. We thus need a function that calculates the value of a candidate.
For this problem, it's simply the sum of the numbers in the sequence.

In [1]:
def value(numbers: list) -> int:
    """Return the value of the candidate: the sum of the numbers."""
    total = 0
    for number in numbers:
        total = total + number
    return total

With respect to the earlier solution for the constraint satisfaction problem,
I need to make two changes. First,
instead of appending each solution found to a sequence,
the backtracking algorithm must receive the current best solution as input and
update it when it finds a better solution.
Second, the main function must compute some initial best solution and pass it
to the backtracking function.
Let's do these changes in turn.

### 22.4.2 Keep the best

The backtracking algorithm must compare each found solution to the current best
and update the latter if the new solution is better.
If it's a minimisation problem, better means having a lower value,
otherwise better means higher.

To avoid recomputing the best solution's value every time it's compared to
a new solution, I will trade space for time and keep a solution–value pair
instead of just the best solution. Usually I represent a pair with a tuple,
but to be able to update the pair, I use a list of length two with
[constants to name the indices](../04_Iteration/04_5_tuples.ipynb#4.5.3-Tables).

In [2]:
SOLUTION = 0
VALUE = 1

I could instead have defined a class with two data fields,
similar to the node classes for
[linked lists](../06_Implementing/06_7_linked_list.ipynb#6.7.3-The-LinkedSequence-class) and
[binary trees](../16_Trees/16_1_binary.ipynb#16.1.2-ADT-and-data-structure).
I'll leave that as an optional exercise, if you wish to practise using classes.

The backtracking function is largely the same as before.
The `solutions` parameter is replaced by the best solution–value pair,
which is updated when the current candidate is a better solution.

I add some print statements to later follow the algorithm's actions.

In [3]:
def extend(candidate: list, extensions: set, n: int, best: list) -> None:
    """Update best if needed, and extend candidate."""
    print("Visiting node", candidate, extensions)
    if satisfies_range(candidate, n):
        candidate_value = value(candidate)
        if candidate_value < best[VALUE]:
            print("New best", candidate, "with value", candidate_value)
            best[SOLUTION] = candidate
            best[VALUE] = candidate_value
    for item in extensions:
        if can_extend(item, candidate):
            extend(candidate + [item], extensions - {item}, n, best)

The current best solution must be initialised before starting the search.
Since the search updates the current best every time a better one is found,
the initial best can be *any* solution,
preferably one that is easy to construct.

For this problem, sequence (1, 2, ..., *n*) is a solution:
it satisfies both constraints.

In [4]:
def best_sequence(n: int) -> list:
    """Return a lowest sum sequence that satisfies both constraints."""
    candidate = []
    extensions = set(range(1, n + 1))
    solution = list(range(1, n + 1))
    best = [solution, value(solution)]
    extend(candidate, extensions, n, best)
    return best

The constraint checking functions are as before.

In [5]:
def satisfies_range(candidate: list, n: int) -> bool:
    """Check if first and last numbers in candidate are at least n/2 apart.

    Preconditions: candidate is a list of integers; n > 2
    """
    return len(candidate) > 1 and abs(candidate[0] - candidate[-1]) >= n / 2


def can_extend(item: int, candidate: list) -> bool:
    """Check if extending candidate with item can lead to a solution."""
    # the number and the index where it will be put must have different parity
    return item % 2 != len(candidate) % 2


best_sequence(4)

Visiting node [] {1, 2, 3, 4}
Visiting node [1] {2, 3, 4}
Visiting node [1, 2] {3, 4}
Visiting node [1, 2, 3] {4}
New best [1, 2, 3] with value 6
Visiting node [1, 2, 3, 4] set()
Visiting node [1, 4] {2, 3}
New best [1, 4] with value 5
Visiting node [1, 4, 3] {2}
Visiting node [1, 4, 3, 2] set()
Visiting node [3] {1, 2, 4}
Visiting node [3, 2] {1, 4}
Visiting node [3, 2, 1] {4}
Visiting node [3, 2, 1, 4] set()
Visiting node [3, 4] {1, 2}
Visiting node [3, 4, 1] {2}
Visiting node [3, 4, 1, 2] set()


[[1, 4], 5]

As desired, the result is the sequence (1, 4) with lowest sum 5.

This version visits 15 nodes. Can we further prune the search space?
(All together now: oh yes, we can!)

### 22.4.3 Avoid worse candidates

When using backtracking for an optimisation problem,
we must think whether we can avoid generating candidates that won't lead to
a better solution than the current one.

In this problem, as a sequence is extended, its sum grows
because all extensions are positive numbers.
When a sequence reaches or surpasses the sum of the current best
solution, we know this sequence can't lead to a better solution,
with a lower sum, so we can stop extending it.

To do this check, the `can_extend` function needs another parameter:
the current best.

In [6]:
def can_extend(item: int, candidate: list, best: list) -> bool:
    """Check if item can extend candidate to a better solution than best."""
    return item % 2 != len(candidate) % 2 and item + value(candidate) < best[VALUE]

Because of the extra parameter, we must change its call in `extend`.

In [7]:
def extend(candidate: list, extensions: set, n: int, best: list) -> None:
    """Update best if a better extension of candidate is found."""
    print("Visiting node", candidate, extensions)
    if satisfies_range(candidate, n):
        candidate_value = value(candidate)
        if candidate_value < best[VALUE]:
            print("New best", candidate, "with value", candidate_value)
            best[SOLUTION] = candidate
            best[VALUE] = candidate_value
    for item in extensions:
        if can_extend(item, candidate, best):  # changed line
            extend(candidate + [item], extensions - {item}, n, best)


best_sequence(4)

Visiting node [] {1, 2, 3, 4}
Visiting node [1] {2, 3, 4}
Visiting node [1, 2] {3, 4}
Visiting node [1, 2, 3] {4}
New best [1, 2, 3] with value 6
Visiting node [1, 4] {2, 3}
New best [1, 4] with value 5
Visiting node [3] {1, 2, 4}


[[1, 4], 5]

As you can see, only six of the previous 15 nodes are created and visited.
Once a solution with sum 5 is found, partial candidates starting with 3 aren't
generated because number 3 can only be followed by 2 or 4, both leading to
sequences with sum at least 5 and thus not improving on the current best.

Is this the best (pun intended) we can do to prune the search space?
(All together: oh no, it isn't!)

### 22.4.4 Start well

I mentioned earlier that we can start with any solution because the search
continuously improves on it until it finds the best solution. However,
now that the algorithm uses the best solution to not generate candidates
leading to equally good or worse solutions,
we should start with an initial solution close to a best one,
to further prune the search space.
The trick is to think of a good solution that is easy to construct.

Based on the fact that (1, 4) is a best solution for *n* = 4,
I choose (1, *n*) or (1, 2, *n*) as the initial solution,
depending on whether *n* is even or odd. (Remember that *n* > 2.)

In [8]:
def best_sequence(n: int) -> list:
    """Return a lowest sum sequence that satisfies both constraints."""
    candidate = []
    extensions = set(range(1, n + 1))
    if n % 2 == 0:
        solution = [1, n]
    else:
        solution = [1, 2, n]
    best = [solution, value(solution)]
    extend(candidate, extensions, n, best)
    return best


best_sequence(4)

Visiting node [] {1, 2, 3, 4}
Visiting node [1] {2, 3, 4}
Visiting node [1, 2] {3, 4}
Visiting node [3] {1, 2, 4}


[[1, 4], 5]

The final version only visits four of the initial version's 15 nodes:
the search space has been reduced by over 70%!

To sum up, backtracking can solve optimisation problems by
keeping track of the current best solution and its value, and by
updating both when a better solution is found.
For optimisation problems where extending a candidate worsens its value,
e.g. makes the value larger but it's a minimisation problem,
the search space can be pruned by stopping extending a candidate when
its value is equal to or worse than the current best. Further pruning
can be obtained by constructing an initial solution close to a best one.

⟵ [Previous section](22_3_trackword.ipynb) | [Up](22-introduction.ipynb) | [Next section](22_5_tsp.ipynb) ⟶